In [1]:
import pandas as pd
import sys
import numpy as np


DATA_PATH = '../data/data.csv.zip'

sys.path.append('../src')
from utils import *

In [2]:
download_dataset(DATA_PATH)

In [3]:
data = pd.read_csv(DATA_PATH, compression='zip')

In [4]:
data.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,No,16.60,Yes,No,No,3.0,30.0,No,Female,55-59,White,Yes,Yes,Very good,5.0,Yes,No,Yes
1,No,20.34,No,No,Yes,0.0,0.0,No,Female,80 or older,White,No,Yes,Very good,7.0,No,No,No
2,No,26.58,Yes,No,No,20.0,30.0,No,Male,65-69,White,Yes,Yes,Fair,8.0,Yes,No,No
3,No,24.21,No,No,No,0.0,0.0,No,Female,75-79,White,No,No,Good,6.0,No,No,Yes
4,No,23.71,No,No,No,28.0,0.0,Yes,Female,40-44,White,No,Yes,Very good,8.0,No,No,No


In [5]:
with open('../data/description.txt') as desc_file:
    lines = desc_file.readlines()
lines = [line.strip() for line in lines if line != '\n']
lines = np.array(lines).reshape(len(lines) // 2, 2)
lines = pd.DataFrame(lines, columns=['feature_name', 'feature_description']).set_index('feature_name')

In [6]:
data_description = pd.concat([lines, data.nunique(), data.dtypes], axis=1)
data_description.columns = ['feature_description', 'feature_nunique', 'feature_dtype']

In [24]:
isinstance(data.dtypes.tolist()[0], object)

True

In [29]:
data_description

,feature_description,feature_nunique,feature_dtype
HeartDisease,Respondents that have ever reported having cor...,2,object
BMI,Body Mass Index (BMI),3604,float64
Smoking,Have you smoked at least 100 cigarettes in you...,2,object
AlcoholDrinking,Heavy drinkers (adult men having more than 14 ...,2,object
Stroke,(Ever told) (you had) a stroke?,2,object
PhysicalHealth,"Now thinking about your physical health, which...",31,float64
MentalHealth,"Thinking about your mental health, for how man...",31,float64
DiffWalking,Do you have serious difficulty walking or clim...,2,object
Sex,Are you male or female?,2,object
AgeCategory,Fourteen-level age category,13,object


In [34]:
data.HeartDisease.value_counts()

No     292422
Yes     27373
Name: HeartDisease, dtype: int64

Видно, что выборка является несбалансированной относительно целевой переменной. </br>
Да и само название целевой переменной подсказывает, что нам критичнее сделать ошибку FN, чем FP. </br>
Отсюда можно сделать вывод, что нам важно максимизировать полноту(recall). </br>
С другой стороны просто так этого делать нельзя, т.к. алгоритм научится предскащывать только 1. </br>
Соотвественно перейдём к F1 в качестве метрики для валидации.

In [38]:
bool_cols = data_description[data_description.feature_nunique == 2].index.tolist()
num_cols = data_description[data_description.feature_nunique >= 15].index.tolist()
cat_cols = data_description[(data_description.feature_nunique > 2) & 
                             (data_description.feature_nunique < 15)].index.tolist()

In [44]:
for col in cat_cols:
    print(data[col].value_counts())

65-69          34151
60-64          33686
70-74          31065
55-59          29757
50-54          25382
80 or older    24153
45-49          21791
75-79          21482
18-24          21064
40-44          21006
35-39          20550
30-34          18753
25-29          16955
Name: AgeCategory, dtype: int64
White                             245212
Hispanic                           27446
Black                              22939
Other                              10928
Asian                               8068
American Indian/Alaskan Native      5202
Name: Race, dtype: int64
No                         269653
Yes                         40802
No, borderline diabetes      6781
Yes (during pregnancy)       2559
Name: Diabetic, dtype: int64
Very good    113858
Good          93129
Excellent     66842
Fair          34677
Poor          11289
Name: GenHealth, dtype: int64


Краткий итог про признаки:
* Часть надо будет перекодировать в 0-1.
* Возраст, диабет и общее здоровье по факту явлюятся порядковыми признаками - соотвественно, при их энкодинге необходимо этот порядок соблюдать.
* Чисто категориальный признак только один - это раса.

In [9]:
import pandas as pd
from sklearn.preprocessing import OrdinalEncoder


df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['b', 'a', 'c'],
                   'C': [1, 2, 3]})

In [10]:
pd.get_dummies(df, prefix=['col1', 'col2'])

,C,col1_a,col1_b,col2_a,col2_b,col2_c
0,1,1,0,0,1,0
1,2,0,1,1,0,0
2,3,1,0,0,0,1
